In [3]:
import tkinter as tk
import random

def draw_points():
    try:
        num_points = int(entry.get())
        canvas.delete("all")  # 캔버스 초기화

        for _ in range(num_points):
            x = random.randint(0, canvas.winfo_width())
            y = random.randint(0, canvas.winfo_height())
            canvas.create_oval(x-3, y-3, x+3, y+3, fill="blue", outline="blue")

    except ValueError:
        result_label.config(text="숫자를 입력하세요.", fg="red")

# 메인 윈도우 설정
root = tk.Tk()
root.title("무작위 점 그리기")

# 입력 필드 및 버튼 설정
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

tk.Label(frame, text="점의 개수:").grid(row=0, column=0)
entry = tk.Entry(frame)
entry.grid(row=0, column=1)

draw_button = tk.Button(frame, text="점 그리기", command=draw_points)
draw_button.grid(row=1, columnspan=2, pady=5)

result_label = tk.Label(frame, text="", fg="red")
result_label.grid(row=2, columnspan=2)

# 캔버스 설정
canvas = tk.Canvas(root, bg="white", width=500, height=500)
canvas.pack(padx=10, pady=10)

# GUI 루프 시작
root.mainloop()


In [13]:
import tkinter as tk
import random
import math
import copy
import timeit

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def euclidean_distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

def bruteForce(P, n):
    min_val = float('inf')
    closest_pair = None
    for i in range(n):
        for j in range(i + 1, n):
            if euclidean_distance(P[i], P[j]) < min_val:
                min_val = euclidean_distance(P[i], P[j])
                closest_pair = (P[i], P[j])
    return closest_pair

def stripClosest(strip, size, d):
    min_val = d
    closest_pair = None
    for i in range(size):
        j = i + 1
        while j < size and (strip[j].y - strip[i].y) < min_val:
            dist_ij = euclidean_distance(strip[i], strip[j])
            if dist_ij < min_val:
                min_val = dist_ij
                closest_pair = (strip[i], strip[j])
            j += 1
    return closest_pair

def closestUtil(P, Q, n):
    if n <= 3:
        return bruteForce(P, n)

    mid = n // 2
    midPoint = P[mid]
    Pl = P[:mid]
    Pr = P[mid:]

    Ql = [p for p in Q if p.x <= midPoint.x]
    Qr = [p for p in Q if p.x > midPoint.x]

    (p1, q1) = closestUtil(Pl, Ql, len(Pl))
    (p2, q2) = closestUtil(Pr, Qr, len(Pr))

    d = min(euclidean_distance(p1, q1), euclidean_distance(p2, q2))
    min_pair = (p1, q1) if euclidean_distance(p1, q1) < euclidean_distance(p2, q2) else (p2, q2)

    strip = [p for p in Q if abs(p.x - midPoint.x) < d]
    strip_closest_pair = stripClosest(strip, len(strip), d)
    
    if strip_closest_pair:
        (p, q) = strip_closest_pair
        if euclidean_distance(p, q) < d:
            min_pair = (p, q)
    
    return min_pair

def closest(P, n):
    P.sort(key=lambda point: point.x)
    Q = copy.deepcopy(P)
    Q.sort(key=lambda point: point.y)
    return closestUtil(P, Q, n)

def draw_points():
    try:
        num_points = int(entry.get())
        canvas.delete("all")  # 캔버스 초기화
        points.clear()  # 점 목록 초기화
        canvas_points.clear()  # 화면에 그린 점 목록 초기화

        for _ in range(num_points):
            x = random.randint(0, canvas.winfo_width())
            y = random.randint(0, canvas.winfo_height())
            point = Point(x, y)
            canvas.create_oval(x-3, y-3, x+3, y+3, fill="blue", outline="blue")
            points.append(point)
            canvas_points.append((x, y))

    except ValueError:
        result_label.config(text="숫자를 입력하세요.", fg="red")

def find_closest_pair_bruteforce(points):
    min_dist = float('inf')
    closest_pair = None
    num_points = len(points)
    for i in range(num_points):
        for j in range(i + 1, num_points):
            dist = euclidean_distance(points[i], points[j])
            if dist < min_dist:
                min_dist = dist
                closest_pair = (points[i], points[j])
    return closest_pair

def find_closest_pair_divide_and_conquer(points):
    def find_closest_pair(px, py):
        if len(px) <= 3:
            return find_closest_pair_bruteforce(px)
        
        mid = len(px) // 2
        midpoint = px[mid].x
        
        Qx = px[:mid]
        Rx = px[mid:]
        Qy = [p for p in py if p.x <= midpoint]
        Ry = [p for p in py if p.x > midpoint]
        
        (p1, q1) = find_closest_pair(Qx, Qy)
        (p2, q2) = find_closest_pair(Rx, Ry)
        
        min_dist = float('inf')
        closest_pair = None
        
        if p1 and q1:
            dist = euclidean_distance(p1, q1)
            if dist < min_dist:
                min_dist = dist
                closest_pair = (p1, q1)
        
        if p2 and q2:
            dist = euclidean_distance(p2, q2)
            if dist < min_dist:
                min_dist = dist
                closest_pair = (p2, q2)

        strip = [p for p in py if abs(p.x - midpoint) < min_dist]
        strip_closest_pair = stripClosest(strip, len(strip), min_dist)
        
        if strip_closest_pair:
            (p, q) = strip_closest_pair
            if euclidean_distance(p, q) < min_dist:
                closest_pair = (p, q)
        
        return closest_pair

    points_sorted_x = sorted(points, key=lambda p: p.x)
    points_sorted_y = sorted(points, key=lambda p: p.y)
    
    return find_closest_pair(points_sorted_x, points_sorted_y)

def show_bruteforce_result():
    if len(points) < 2:
        result_label.config(text="점이 충분하지 않습니다.", fg="red")
        return
    
    start_time = timeit.default_timer()
    pair = find_closest_pair_bruteforce(points)
    elapsed = timeit.default_timer() - start_time
    
    if pair:
        (p1, p2) = pair
        canvas.create_line(p1.x, p1.y, p2.x, p2.y, fill="red", width=2)
        result_label.config(text=f"브루트포스: ({p1.x}, {p1.y})와 ({p2.x}, {p2.y})")
        time_label.config(text=f"브루트포스 시간: {elapsed:.6f}초", fg="black")
    else:
        result_label.config(text="쌍을 찾을 수 없습니다.", fg="red")
        time_label.config(text="브루트포스 시간: 측정 실패", fg="red")

def show_divide_and_conquer_result():
    if len(points) < 2:
        result_label.config(text="점이 충분하지 않습니다.", fg="red")
        return
    
    start_time = timeit.default_timer()
    pair = find_closest_pair_divide_and_conquer(points)
    elapsed = timeit.default_timer() - start_time
    
    if pair:
        (p1, p2) = pair
        canvas.create_line(p1.x, p1.y, p2.x, p2.y, fill="green", width=2)
        result_label.config(text=f"분할 정복: ({p1.x}, {p1.y})와 ({p2.x}, {p2.y})")
        time_label.config(text=f"분할 정복 시간: {elapsed:.6f}초", fg="black")
    else:
        result_label.config(text="쌍을 찾을 수 없습니다.", fg="red")
        time_label.config(text="분할 정복 시간: 측정 실패", fg="red")

# 메인 윈도우 설정
root = tk.Tk()
root.title("무작위 점 그리기")

# 입력 필드 및 버튼 설정
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

tk.Label(frame, text="점의 개수:").grid(row=0, column=0)
entry = tk.Entry(frame)
entry.grid(row=0, column=1)

draw_button = tk.Button(frame, text="점 그리기", command=draw_points)
draw_button.grid(row=1, columnspan=2, pady=5)

bruteforce_button = tk.Button(frame, text="브루트포스", command=show_bruteforce_result)
bruteforce_button.grid(row=2, column=0, pady=5)

divide_conquer_button = tk.Button(frame, text="분할 정복", command=show_divide_and_conquer_result)
divide_conquer_button.grid(row=2, column=1, pady=5)

result_label = tk.Label(frame, text="", fg="red")
result_label.grid(row=3, columnspan=2)

time_label = tk.Label(frame, text="", fg="black")
time_label.grid(row=4, columnspan=2)

# 캔버스 설정
canvas = tk.Canvas(root, bg="white", width=500, height=500)
canvas.pack(padx=10, pady=10)

# 점 리스트 초기화
points = []
canvas_points = []

# GUI 루프 시작
root.mainloop()
